In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import library

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import mlxtend
import sklearn.cluster as cluster
import sklearn.neighbors
import sklearn.metrics as metrics
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score, precision_recall_curve
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
import warnings
warnings.filterwarnings('ignore')

In [25]:
train_data = pd.read_csv('../input/emotions-dataset-for-nlp/train.txt',names=['sentence','emotion'],header=None, sep=';')
test_data = pd.read_csv('../input/emotions-dataset-for-nlp/test.txt',names=['sentence','emotion'],header=None, sep=';')
val_data= pd.read_csv('../input/emotions-dataset-for-nlp/val.txt',names=['sentence','emotion'],header=None, sep=';')
data = pd.concat([train_data,test_data, val_data])
print('Total data:',data.shape)


In [26]:
train_data.head()

In [27]:
data.isnull().sum()

In [28]:
data.emotion.unique()

In [29]:
data['emotion'].value_counts(normalize=True)


In [30]:
data.info

In [31]:
#for convert words to numerical data by mapping 
#emotion_mapping={'sadness':1,'anger':2,'love':3,'surprise':4,"fear":5,"joy":6}

#data['emotion']=data['emotion'].map(emotion_mapping)
#data.head()

In [32]:
data['length'] = data['sentence'].apply(len) # to add length column 
data.head()

In [33]:
# convert the emotions to binary labels. love and joy emotions are "not-stressed ==1", and sadness, anger, fear, and surprise are "stressed == 0".
data['label']=data['emotion'].replace({'joy': "not-stressed" , 'love': "not-stressed", 
                                   'sadness': "stressed", 'anger': "stressed", 'fear': "stressed",'surprise': "stressed"})
data.head()

In [47]:
data.label.value_counts()

In [48]:
#for convert words to numerical data by mapping 
label_mapping={'stressed':1,'not-stressed':2}

data['label']=data['label'].map(label_mapping)
data.head()

In [49]:
data.duplicated().sum()   #to know how many duplicated 

In [50]:
data=data.drop_duplicates()   #to delete duplicate
data = data.reset_index(drop=True)


In [51]:
data.shape[0]

Text Preprocessing¶


In [52]:
from tqdm import tqdm
import re
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def decontracted(phrase):
    """
    We first define a function to expand the contracted phrase into normal words
    """
    # specific
    phrase = re.sub(r"wont", "will not", phrase)
    phrase = re.sub(r"wouldnt", "would not", phrase)
    phrase = re.sub(r"shouldnt", "should not", phrase)
    phrase = re.sub(r"couldnt", "could not", phrase)
    phrase = re.sub(r"cudnt", "could not", phrase)
    phrase = re.sub(r"cant", "can not", phrase)
    phrase = re.sub(r"dont", "do not", phrase)
    phrase = re.sub(r"doesnt", "does not", phrase)
    phrase = re.sub(r"didnt", "did not", phrase)
    phrase = re.sub(r"wasnt", "was not", phrase)
    phrase = re.sub(r"werent", "were not", phrase)
    phrase = re.sub(r"havent", "have not", phrase)
    phrase = re.sub(r"hadnt", "had not", phrase)

    # general
    phrase = re.sub(r"n\ t", " not", phrase)
    #phrase = re.sub(r"\re", " are", phrase)
    phrase = re.sub(r"\ s ", " is ", phrase) # prime 
    phrase = re.sub(r"\ d ", " would ", phrase)
    phrase = re.sub(r"\ ll ", " will ", phrase)
    phrase = re.sub(r"\dunno", "do not ", phrase)
    phrase = re.sub(r"ive ", "i have ", phrase)
    phrase = re.sub(r"im ", "i am ", phrase)
    phrase = re.sub(r"i m ", "i am ", phrase)
    phrase = re.sub(r" w ", " with ", phrase)
    
    return phrase
    
def clean_text(data):
    """
    Clean the review texts
    """
    cleaned_review = []

    for review_text in tqdm(data['sentence']):
        
        # expand the contracted words
        review_text = decontracted(review_text)
        #remove html tags
        review_text = BeautifulSoup(review_text, 'lxml').get_text().strip() # re.sub(r'<.*?>', '', text)
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        #remove url 
        review_text = re.sub(r'https?://\S+|www\.\S+', '', review_text)
        
        #Removing punctutation, string.punctuation in python consists of !"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`
       # review_text = review_text.translate(str.maketrans('', '', string.punctuation))
        # ''.join([char for char in movie_text_data if char not in string.punctuation])
        
        # remove emails
        review_text = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", '', review_text)
    
        cleaned_review.append(review_text)

    return cleaned_review  

data['cleaned_sentence'] = clean_text(data)
data.head() 

In [53]:
#lemmmentation
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('punkt')
def remove_stopwords(phrase):
    remove_sw = []
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')
    stop_words = stopwords.words('english')
    
    for review_text in tqdm(phrase):
        tokens = word_tokenize(review_text)
        tokens = [word for word in tokens if not word in stop_words]
        remove_sw.append(tokens)
    return remove_sw

data['cleaned_sentence'] = remove_stopwords(data['cleaned_sentence'])
data.head()

In [54]:
#stemming for extract the actual meaning of the words
from nltk.stem import PorterStemmer

def stemming(phrase):
    stemmer = PorterStemmer()
    stem_output=[]
    stemmed=[]
    for review_text in tqdm(phrase):
        stemmed = [stemmer.stem(word) for word in review_text]
        stem_output.append(stemmed)
    return stem_output

data['cleaned_sentence'] = stemming(data['cleaned_sentence'])
data['cleaned_sentence'].head()

In [55]:
def to_sentence(phrase):
    sentence=[]
    for words in tqdm(phrase):
        sentence.append((" ").join(words))
    return sentence
data['cleaned_sentence']=to_sentence(data['cleaned_sentence'])
data['cleaned_sentence'].head()

In [57]:
# convert the cleaned sentences to vectors
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
vectorizer = CountVectorizer(stop_words='english', max_df=0.5, min_df=3, ngram_range=(1,1),tokenizer = token.tokenize)
x = vectorizer.fit_transform(data.cleaned_sentence)
y = data.label.values

print("X.shape : ",x.shape)
print("y.shape : ",y.shape)


split data "train+validation+test" to train and test

In [58]:


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20,shuffle=True, random_state=42)
x_train

In [45]:
x_train.shape

In [59]:
# fit a logistic regression classifier on the training data use default settings
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(x_train, y_train)

# make prediction on testing data
y_pred_test_lr = lr_clf.predict(x_test)
y_predprob_lr = lr_clf.predict_proba(x_test)
matrix_lr = confusion_matrix(y_test,y_pred_test_lr)
print(classification_report(y_test, y_pred_test_lr))
print("\nAccuracy for Logistic Regression model:",metrics.accuracy_score(y_test, y_pred_test_lr))
print("\n")
y_predict = lr_clf.predict(x_test)
matrix_display = ConfusionMatrixDisplay(matrix_lr).plot()

In [62]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)
mlp.fit(x_train, y_train)

In [63]:
predictions = mlp.predict(x_test)

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))